In [13]:
# Util
import random

# Model
import agentpy as ap
import matplotlib.animation

# Visualization
import matplotlib.pyplot as plt
import IPython

VALID_MOVES = {
    'RIGHT': (0, 1),
    'DOWN': (1, 0)
}

GREEN = 1
YELLOW = 2
RED = 3
BLUE = 4


class Car(ap.Agent):
    def setup(self):
        self.dir = random.choice(list(VALID_MOVES.keys()))
        self.color = BLUE
        self.type = 'CAR'

    def get_dir(self):
        return VALID_MOVES[self.dir] if random.uniform(0, 1) > 0.3 else (0, 0)

    def get_dir_name(self):
        return self.dir


class TrafficLight(ap.Agent):
    def setup(self):
        self.color = YELLOW

        
class Interseccion(ap.Model):

    def add_car(self):
        self.new_car = ap.AgentList(self, int(1), Car)

        start_positions = {
            'LEFT': (self.v_pos, 0),
            'UP': (0, self.h_pos)
        }

        dir_name = self.new_car.get_dir_name()

        if dir_name == 'RIGHT':
            position = start_positions['LEFT']
        elif dir_name == 'DOWN':
            position = start_positions['UP']

        self.grid.add_agents(self.new_car, [position])

    def setup(self):
        # Create streets
        street_length = self.p.size
        self.h_pos = street_length // 2
        self.v_pos = street_length // 2

        # Create grid (calles)
        self.grid = ap.Grid(self, (self.p.size, self.p.size), track_empty=True)

        # Step counter
        self.step_count = 0

    def step(self):
        self.add_car()
        #agent_list = ap.AgentList(self, 1, Car)

        for agent in self.grid.agents:
            if agent.type == 'CAR':
                self.grid.move_by(agent, agent.get_dir())

        if self.p.time == self.step_count:
            self.stop()

        self.step_count += 1

    def end(self):
        pass


def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('color')
    color_dict = {1: '#00ff00', 2: '#ffff00', 3: '#ff0000', 4: '#eeeeee', None: '#ffffff'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of street intersection\n"
                 f"Time-step: {model.t}, Trees left: ")

parameters = {
    'size': 10,
    'n_cars': 2,
    'time': 10
}

fig, ax = plt.subplots()
model = Interseccion(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

